In [2]:
import evaluate
import numpy as np
from tqdm import tqdm
import json
from nltk.tokenize import RegexpTokenizer
bleu_metric = evaluate.load("sacrebleu")

rouge_metric = evaluate.load("rouge")
sari_metric = evaluate.load("sari")

def tokenize_sentence(arg):
    encoded_arg =tokenizer(arg)
    return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

def metrics_func(eval_arg):
    print(len(eval_arg[0]),len(eval_arg[1]),len(eval_arg[2]))
    print(len(eval_arg))
    text_inputs = eval_arg[0]
    text_preds = eval_arg[1]
    text_labels = eval_arg[2]
    texts_bleu =[text.strip() for text in text_preds]
    labels_bleu = [[text.strip()] for text in text_labels[0]]
    result = bleu_metric.compute(predictions=texts_bleu, references=text_labels)
    return result["score"],sari_metric.compute(
        predictions=text_preds,
        references=text_labels,
        sources=text_inputs,
    )['sari']

# %%
from torch.utils.data import DataLoader
#tokenizer = MT5Tokenizer.from_pretrained("./mt5")

#load json
with open('../../data/10/dissim_dict.json') as f:
    dissim_dict = json.load(f)

with open('../../data/10/input.txt') as f:
    inputs = f.readlines()

with open('../../data/10/labels.txt') as f:
    labels = f.readlines()

metrics =[]
inpu = []
cands = []
lab = []
for i in tqdm(inputs):
        inpu.extend([i.strip()]*len(dissim_dict[i.strip()]))
        cands.extend(dissim_dict[i.strip()])
        lab.extend([[labels[inputs.index(i)]]]*len(dissim_dict[i.strip()]))
        #print(len(inpu),len(cands),len(lab))
        eval_arg = [inpu,cands,lab]
        #metric = metrics_func(eval_arg)
        #print(metric)
        #print("++++++++++++++++++++++++++++++")
        #metrics.append(metric)

print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)

# def average_metric(metrics):
#     rouge = 0
#     rouge2 = 0
#     rougeL = 0
#     rougeLsum = 0
#     bleu = 0
#     sari =0
#     for metric in metrics:
#         # rouge += metric[0]['rouge1']
#         # rouge2 += metric[0]['rouge2']
#         # rougeL += metric[0]['rougeL']
#         # rougeLsum += metric[0]['rougeLsum']
#         bleu += metric[0]
#         sari += metric[1]

#     return bleu/len(metrics),sari/len(metrics)
      

# print("t5_small")
# scores = average_metric(metrics)
# print("rouge:",scores[0])
# print("rouge2:",scores[1])
# print("rougeL:",scores[2])
# print("rougeLsum:",scores[3])
# print("bleu:",scores[4])


# # %%
# from torch.utils.data import DataLoader

# Predict with test data (first 5 rows)
# sample_dataloader = DataLoader(
#   test.with_format("torch"),
#   collate_fn=DataCollatorForSeq2Seq(tokenizer, model=model),
#   batch_size=5)
# for batch in sample_dataloader:
#   with torch.no_grad():
#     preds = model.generate(
#       batch["input_ids"],
#       num_beams=15,
#       num_return_sequences=1,
#       no_repeat_ngram_size=1,
#       remove_invalid_values=True,
#       max_length=128,
#     )
#   labels = batch["labels"]
#   inputs = batch["input_ids"]
#   break

# # Replace -100 (see above)
# inputs = np.where(inputs != -100, inputs, tokenizer.pad_token_id)
# labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

# # Convert id tokens to text
# text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
# text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
# text_inputs = tokenizer.batch_decode(inputs, skip_special_tokens=True)
# #print(bleu_score(list(text_labels.split()),list(text_preds.split())))

# # Show result
# print("***** Input's Text *****")
# print(text_inputs[2])
# print("***** summmary (True Value) *****")
# print(text_labels[2])
# print("***** summary (Generated Text) *****")
# print(text_preds[2])

# # %%
# for i in range(5):
#     print("***** Input's Text *****")
#     print(text_inputs[i])
#     print("***** summarry (True Value) *****")
#     print(text_labels[i])
#     print("***** summary (Generated Text) *****")
#     print(text_preds[i])




100%|██████████| 33801/33801 [00:18<00:00, 1863.94it/s]


83258 83258 83258
83258 83258 83258
3


That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.


(8.680953594011816, 32.834411652484036)
